In [3]:
import time
import pandas as pd
import numpy as np
import math
from math import cos, asin, sqrt
import gc
import os.path


pd.set_option('display.max_columns', None)

In [4]:
start = time.time()
#########################controlar la excepcion si no esta el fichero creado-----------------------------------------
vuelos = pd.read_csv('vuelos.csv', sep=',',low_memory=False, )
vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

end = time.time()
print(end - start)

5.47900009155


In [5]:
start = time.time()

if vuelos.empty:
    #hay que tener cuidado de no machacar nada.
    vuelos = pd.read_csv('vuelosDeparted.csv', sep=',',low_memory=False, )
    vuelos.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

    vuelos["TMIN_o"]= ''
    vuelos["TMIN_d"]= ''
    vuelos["TMAX_o"]= ''
    vuelos["TMAX_d"]= ''
    vuelos["TAVG_o"]= ''
    vuelos["TAVG_d"]= ''
    vuelos["SNOW_o"]= ''
    vuelos["SNOW_d"]= ''
    vuelos["PRCP_o"]= ''
    vuelos["PRCP_d"]= ''
    vuelos["SNWD_o"]= ''
    vuelos["SNWD_d"]= ''
    vuelos["ACMC_o"]= ''
    vuelos["ACMC_d"]= ''
    vuelos["ACSC_o"]= ''
    vuelos["ACSC_d"]= ''
    vuelos["AWDR_o"]= ''
    vuelos["AWDR_d"]= ''
    vuelos["AWND_o"]= ''
    vuelos["AWND_d"]= ''
    vuelos["EVAP_o"]= ''
    vuelos["EVAP_d"]= ''
    vuelos["FRTH_o"]= ''
    vuelos["FRTH_d"]= ''
    vuelos["TSUN_o"]= ''
    vuelos["TSUN_d"]= ''
    vuelos["WDMV_o"]= ''
    vuelos["WDMV_d"]= ''


    vuelos.to_csv('vuelos.csv', sep=',', index=False)

end = time.time()
print(end - start)

0.000999927520752


In [6]:
start = time.time()
##Leemos el fichero de estaciones de noaa

stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()
    
#Extract the data from file
stations2 = stationstxt.split("\n")
#Remove last line
stations2 = stations2[:-1]
stations2 = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations2)

end = time.time()
print(end - start)

0.496999979019


# Leemos datos del tiempo de noaa

In [119]:
start = time.time()

year = '2015'

weatherdf = pd.read_csv(year+".csv",header=None)
weatherdf.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]

##nos quedamos con las latitudes de las estaciones.
stationsdf = pd.DataFrame(stations2)
stationsdf.columns = ["id","lat","lng","name"]
lats = stationsdf[['lat','lng']]

#funciones para calcular la distancia entre los aeropuertos y estacion metereologica mas cercana
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))


#para que la fecha no de problemas al comparar (quizas no este bien del todo)
def tratarFecha(fecha):
    fecha = str(fecha).split(' ')[0]
    fecha = fecha.replace('-','')
    fecha = int(fecha)
    return fecha



del weatherdf['Value3']
del weatherdf['Value4']
del weatherdf['Value5']
del weatherdf['Value2']
weather = weatherdf ##creamos bck



def obtenerEstacion2(row):
    lat = row['board_lat']
    lon = row['board_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lats), coor)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[stationsdf['lat']==EMC[0]]
    aux = stationsdf[stationsdf['lng']==EMC[1]]
    EMC = aux['id']
    s =EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def obtenerEstacion3(row):
    lat = row['off_lat']
    lon = row['off_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lats), coor)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[stationsdf['lat']==EMC[0]]
    aux = stationsdf[stationsdf['lng']==EMC[1]]
    EMC = aux['id']
    s =EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def aplicarEstacionesOrigen(row):
    row['board_stationid']= coordenadasOrigen[(coordenadasOrigen.board_lat == row['board_lat'])&(coordenadasOrigen.board_lon == row['board_lon'])]['stationid'].values[0]
    return row

def aplicarEstacionesDestino(row):
    row['off_stationid']= coordenadasDestino[(coordenadasDestino.off_lat == row['off_lat'])&(coordenadasDestino.off_lon == row['off_lon'])]['stationid'].values[0]
    return row


end = time.time()
print(end - start)

52.9349999428


In [71]:
#aplicamos y obtenemos las estaciones (para mejorar el procesado)
if 'board_stationid' in vuelos:
    pass
else:
    coordenadasOrigen = vuelos[['board_lat','board_lon']]
    coordenadasDestino = vuelos[['off_lat','off_lon']]
    coordenadasOrigen = coordenadasOrigen.drop_duplicates(subset=['board_lat', 'board_lon'])
    coordenadasDestino = coordenadasDestino.drop_duplicates(subset=['off_lat', 'off_lon'])
    coordenadasOrigen = coordenadasOrigen.apply(lambda x:obtenerEstacion2(x),axis = 1)
    coordenadasDestino = coordenadasDestino.apply(lambda x:obtenerEstacion3(x),axis = 1)
    vuelos = vuelos.apply(lambda x: aplicarEstacionesOrigen(x),axis = 1)
    vuelos = vuelos.apply(lambda x: aplicarEstacionesDestino(x),axis = 1)

In [72]:
def tratar2(row):
    
    board_station = row['board_stationid']
    off_station = row['off_stationid']
    fecha = row['actual_time_of_departure']
    fecha_d = row['actual_time_of_arrival'] 
    ##################################
    #primero tratamos las fechas
    fecha = tratarFecha(fecha)
    fecha_d = tratarFecha(fecha_d)
    #########################################################################################################
    data_board = weatherdf[(weatherdf["id"]==board_station)&(weatherdf['date']==fecha)].sort_values("date")
    data_off   = weatherdf[(weatherdf["id"]==off_station)&(weatherdf['date']==fecha_d)].sort_values("date")
    #####################################################
    data_board["Value1"]=data_board["Value1"]/10
    data_board = data_board.reset_index()
    data_off["Value1"]=data_off["Value1"]/10
    data_off = data_off.reset_index()
        #######################################################
    T1 = data_board[data_board['type']=='TMIN']['Value1']
    T2 = data_off[data_off['type']=='TMIN']['Value1']
    T3 = data_board[data_board['type']=='TMAX']['Value1']
    T4 = data_off[data_off['type']=='TMAX']['Value1']
    T5 = data_board[data_board['type']=='TAVG']['Value1']
    T6 = data_off[data_off['type']=='TAVG']['Value1']
    T7 = data_board[data_board['type']=='SNOW']['Value1']
    T8 = data_off[data_off['type']=='SNOW']['Value1']
    T9 = data_board[data_board['type']=='PRCP']['Value1']
    T10 = data_off[data_off['type']=='PRCP']['Value1']
    T11 = data_board[data_board['type']=='SNWD']['Value1']
    T12 = data_off[data_off['type']=='SNWD']['Value1']
    T13 = data_board[data_board['type']=='ACMC']['Value1']
    T14 = data_off[data_off['type']=='ACMC']['Value1']
    T17 = data_board[data_board['type']=='ACSC']['Value1']
    T18 = data_off[data_off['type']=='ACSC']['Value1']
    T21 = data_board[data_board['type']=='AWDR']['Value1']
    T22 = data_off[data_off['type']=='AWDR']['Value1']
    T23 = data_board[data_board['type']=='AWND']['Value1']
    T24 = data_off[data_off['type']=='AWND']['Value1']
    T39 = data_board[data_board['type']=='EVAP']['Value1']
    T40 = data_off[data_off['type']=='EVAP']['Value1']
    T47 = data_board[data_board['type']=='FRTH']['Value1']
    T48 = data_off[data_off['type']=='FRTH']['Value1']
    T75 = data_board[data_board['type']=='TSUN']['Value1']
    T76 = data_off[data_off['type']=='TSUN']['Value1']
    T89 = data_board[data_board['type']=='WDMV']['Value1']
    T90 = data_off[data_off['type']=='WDMV']['Value1']
    
    
    ##############################
    ##tratamos los valores nulos aplicando un valor nulo
    if T1.empty:
        T1 = 9999
    if T2.empty:
        T2 = 9999
    if T3.empty:
        T3 = 9999
    if T4.empty:
        T4 = 9999
    if T5.empty:
        T5 = 9999
    if T6.empty:
        T6 = 9999
    if T7.empty:
        T7 = 9999
    if T8.empty:
        T8 = 9999
    if T9.empty:
        T9 = 9999
    if T10.empty:
        T10 = 9999
    if T11.empty:
        T11 = 9999
    if T12.empty:
        T12 = 9999
    if T13.empty:
        T13 = 9999
    if T14.empty:
        T14 = 9999
    if T17.empty:
        T17 = 9999
    if T18.empty:
        T18 = 9999
    if T21.empty:
        T21 = 9999
    if T22.empty:
        T22 = 9999
    if T23.empty:
        T23 = 9999
    if T24.empty:
        T24 = 9999
    if T39.empty:
        T39 = 9999
    if T40.empty:
        T40 = 9999
    if T47.empty:
        T47 = 9999
    if T48.empty:
        T48 = 9999
    if T75.empty:
        T75 = 9999
    if T76.empty:
        T76 = 9999
    if T89.empty:
        T89 = 9999
    if T90.empty:
        T90 = 9999
      
        
    #actualizamos la fila
    row['TMIN_o']= float(T1)
    row['TMIN_d']= float(T2)
    row['TMAX_o']= float(T3)
    row['TMAX_d']= float(T4)
    row['TAVG_o']= float(T5)
    row['TAVG_d']= float(T6)
    row['SNOW_o']= float(T7)
    row['SNOW_d']= float(T8)
    row['PRCP_o']= float(T9)
    row['PRCP_d']= float(T10)
    row['SNWD_o']= float(T11)
    row['SNWD_d']= float(T12)
    row['ACMC_o']= float(T13)
    row['ACMC_d']= float(T14)
    row['ACSC_o']= float(T17)
    row['ACSC_d']= float(T18)
    row['AWDR_o']= float(T21)
    row['AWDR_d']= float(T22)
    row['AWND_o']= float(T23)
    row['AWND_d']= float(T24)
    row['EVAP_o']= float(T39)
    row['EVAP_d']= float(T40)
    row['FRTH_o']= float(T47)
    row['FRTH_d']= float(T48)
    row['TSUN_o']= float(T75)
    row['TSUN_d']= float(T76)
    row['WDMV_o']= float(T89)
    row['WDMV_d']= float(T90)


    return row

def functionWeatherDay(year, month, day):
    u = int(month+day)
    d = int(year*10000)
    weatherdf = weather##restauramos el bck
    weatherdf = weatherdf[weatherdf['date']-d==u]
    return weatherdf

def functionPlus(year, month, day):    
    start = time.time()
    vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    x = x.apply(lambda x:tratar2(x),axis = 1)
    #para escribir ya
    #variable auxiliar

    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values
    vuelosSinTratar['TMAX_o'] = x.set_index(['index'])['TMAX_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_o']).values
    vuelosSinTratar['TMAX_d'] = x.set_index(['index'])['TMAX_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMAX_d']).values
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values
    vuelosSinTratar['SNOW_o'] = x.set_index(['index'])['SNOW_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_o']).values
    vuelosSinTratar['SNOW_d'] = x.set_index(['index'])['SNOW_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNOW_d']).values
    vuelosSinTratar['PRCP_o'] = x.set_index(['index'])['PRCP_o'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_o']).values
    vuelosSinTratar['PRCP_d'] = x.set_index(['index'])['PRCP_d'].combine_first(vuelosSinTratar.set_index(['index'])['PRCP_d']).values
    vuelosSinTratar['SNWD_o'] = x.set_index(['index'])['SNWD_o'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_o']).values
    vuelosSinTratar['SNWD_d'] = x.set_index(['index'])['SNWD_d'].combine_first(vuelosSinTratar.set_index(['index'])['SNWD_d']).values
    vuelosSinTratar['ACMC_o'] = x.set_index(['index'])['ACMC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_o']).values
    vuelosSinTratar['ACMC_d'] = x.set_index(['index'])['ACMC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACMC_d']).values
    vuelosSinTratar['ACSC_o'] = x.set_index(['index'])['ACSC_o'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_o']).values
    vuelosSinTratar['ACSC_d'] = x.set_index(['index'])['ACSC_d'].combine_first(vuelosSinTratar.set_index(['index'])['ACSC_d']).values
    vuelosSinTratar['AWDR_o'] = x.set_index(['index'])['AWDR_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_o']).values
    vuelosSinTratar['AWDR_d'] = x.set_index(['index'])['AWDR_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWDR_d']).values
    vuelosSinTratar['AWND_o'] = x.set_index(['index'])['AWND_o'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_o']).values
    vuelosSinTratar['AWND_d'] = x.set_index(['index'])['AWND_d'].combine_first(vuelosSinTratar.set_index(['index'])['AWND_d']).values
    vuelosSinTratar['EVAP_o'] = x.set_index(['index'])['EVAP_o'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_o']).values
    vuelosSinTratar['EVAP_d'] = x.set_index(['index'])['EVAP_d'].combine_first(vuelosSinTratar.set_index(['index'])['EVAP_d']).values
    vuelosSinTratar['FRTH_o'] = x.set_index(['index'])['FRTH_o'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_o']).values
    vuelosSinTratar['FRTH_d'] = x.set_index(['index'])['FRTH_d'].combine_first(vuelosSinTratar.set_index(['index'])['FRTH_d']).values
    vuelosSinTratar['TSUN_o'] = x.set_index(['index'])['TSUN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_o']).values
    vuelosSinTratar['TSUN_d'] = x.set_index(['index'])['TSUN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TSUN_d']).values
    vuelosSinTratar['WDMV_o'] = x.set_index(['index'])['WDMV_o'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_o']).values
    vuelosSinTratar['WDMV_d'] = x.set_index(['index'])['WDMV_d'].combine_first(vuelosSinTratar.set_index(['index'])['WDMV_d']).values


    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values
    vuelos['TMAX_o'] = vuelosSinTratar.set_index(['index'])['TMAX_o'].combine_first(vuelos.set_index(['index'])['TMAX_o']).values
    vuelos['TMAX_d'] = vuelosSinTratar.set_index(['index'])['TMAX_d'].combine_first(vuelos.set_index(['index'])['TMAX_d']).values
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values
    vuelos['SNOW_o'] = vuelosSinTratar.set_index(['index'])['SNOW_o'].combine_first(vuelos.set_index(['index'])['SNOW_o']).values
    vuelos['SNOW_d'] = vuelosSinTratar.set_index(['index'])['SNOW_d'].combine_first(vuelos.set_index(['index'])['SNOW_d']).values
    vuelos['PRCP_o'] = vuelosSinTratar.set_index(['index'])['PRCP_o'].combine_first(vuelos.set_index(['index'])['PRCP_o']).values
    vuelos['PRCP_d'] = vuelosSinTratar.set_index(['index'])['PRCP_d'].combine_first(vuelos.set_index(['index'])['PRCP_d']).values
    vuelos['SNWD_o'] = vuelosSinTratar.set_index(['index'])['SNWD_o'].combine_first(vuelos.set_index(['index'])['SNWD_o']).values
    vuelos['SNWD_d'] = vuelosSinTratar.set_index(['index'])['SNWD_d'].combine_first(vuelos.set_index(['index'])['SNWD_d']).values
    vuelos['ACMC_o'] = vuelosSinTratar.set_index(['index'])['ACMC_o'].combine_first(vuelos.set_index(['index'])['ACMC_o']).values
    vuelos['ACMC_d'] = vuelosSinTratar.set_index(['index'])['ACMC_d'].combine_first(vuelos.set_index(['index'])['ACMC_d']).values
    vuelos['ACSC_o'] = vuelosSinTratar.set_index(['index'])['ACSC_o'].combine_first(vuelos.set_index(['index'])['ACSC_o']).values
    vuelos['ACSC_d'] = vuelosSinTratar.set_index(['index'])['ACSC_d'].combine_first(vuelos.set_index(['index'])['ACSC_d']).values
    vuelos['AWDR_o'] = vuelosSinTratar.set_index(['index'])['AWDR_o'].combine_first(vuelos.set_index(['index'])['AWDR_o']).values
    vuelos['AWDR_d'] = vuelosSinTratar.set_index(['index'])['AWDR_d'].combine_first(vuelos.set_index(['index'])['AWDR_d']).values
    vuelos['AWND_o'] = vuelosSinTratar.set_index(['index'])['AWND_o'].combine_first(vuelos.set_index(['index'])['AWND_o']).values
    vuelos['AWND_d'] = vuelosSinTratar.set_index(['index'])['AWND_d'].combine_first(vuelos.set_index(['index'])['AWND_d']).values
    vuelos['EVAP_o'] = vuelosSinTratar.set_index(['index'])['EVAP_o'].combine_first(vuelos.set_index(['index'])['EVAP_o']).values
    vuelos['EVAP_d'] = vuelosSinTratar.set_index(['index'])['EVAP_d'].combine_first(vuelos.set_index(['index'])['EVAP_d']).values
    vuelos['FRTH_o'] = vuelosSinTratar.set_index(['index'])['FRTH_o'].combine_first(vuelos.set_index(['index'])['FRTH_o']).values
    vuelos['FRTH_d'] = vuelosSinTratar.set_index(['index'])['FRTH_d'].combine_first(vuelos.set_index(['index'])['FRTH_d']).values
    vuelos['TSUN_o'] = vuelosSinTratar.set_index(['index'])['TSUN_o'].combine_first(vuelos.set_index(['index'])['TSUN_o']).values
    vuelos['TSUN_d'] = vuelosSinTratar.set_index(['index'])['TSUN_d'].combine_first(vuelos.set_index(['index'])['TSUN_d']).values
    vuelos['WDMV_o'] = vuelosSinTratar.set_index(['index'])['WDMV_o'].combine_first(vuelos.set_index(['index'])['WDMV_o']).values
    vuelos['WDMV_d'] = vuelosSinTratar.set_index(['index'])['WDMV_d'].combine_first(vuelos.set_index(['index'])['WDMV_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [ ]:
#primer ciclo----------------------------poner un bucle para todos los años

#variables de inicio

dia = 1
mes = 1
for mes in range(1,13):
    print 'empiezo el mes', mes
    mesw = str(mes)
    for dia in range(1,32):
        print 'empiezo el dia - ' ,dia
        diaw = str(dia)
        if dia < 10:
            diaw = '0'+diaw

        weatherdf = functionWeatherDay(anyo,mesw,diaw)
        functionPlus(year,mes,dia)

In [73]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)

In [74]:
#para ver el estado del proceso...borrar luego.
vuelosSinTratar = vuelos[vuelos['TAVG_o'].isnull()]
vuelosTratados = vuelos[vuelos['TAVG_o'].notnull()]
print 'sin tratar de todos ', vuelosSinTratar.shape
print 'tratados ', vuelosTratados.shape
x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==2012]
print 'sin tratar totales del año ', x.shape
x = x[x['mesSalida']==mes]
print 'del mes sin tratar ', x.shape
x = x[x['diaSalida']==dia]
print 'vuelos del dia sin tratar ' , x.shape

sin tratar de todos  (0, 88)
tratados  (212698, 88)
sin tratar totales del año  (0, 88)
del mes sin tratar  (0, 88)
vuelos del dia sin tratar  (0, 88)


# segunda fase del tratamiento de datos

In [137]:
print vuelos.shape
print vuelos[vuelos.TMIN_d==9999.0].shape
print vuelos[vuelos.TMIN_o==9999.0].shape
print vuelos[vuelos.TMAX_d==9999.0].shape
print vuelos[vuelos.TMAX_o==9999.0].shape
print vuelos[vuelos.TAVG_d==9999.0].shape
print vuelos[vuelos.TAVG_o==9999.0].shape
print vuelos[vuelos.SNOW_o==9999.0].shape
print vuelos[vuelos.SNOW_d==9999.0].shape
print vuelos[vuelos.PRCP_o==9999.0].shape
print vuelos[vuelos.PRCP_d==9999.0].shape
print vuelos[vuelos.SNWD_o==9999.0].shape
print vuelos[vuelos.SNWD_d==9999.0].shape
print vuelos[vuelos.ACMC_o==9999.0].shape
print vuelos[vuelos.ACMC_d==9999.0].shape
print vuelos[vuelos.ACSC_o==9999.0].shape
print vuelos[vuelos.ACSC_d==9999.0].shape
print vuelos[vuelos.AWDR_o==9999.0].shape
print vuelos[vuelos.AWDR_d==9999.0].shape
print vuelos[vuelos.AWND_o==9999.0].shape
print vuelos[vuelos.AWND_d==9999.0].shape
print vuelos[vuelos.EVAP_o==9999.0].shape
print vuelos[vuelos.EVAP_d==9999.0].shape
print vuelos[vuelos.FRTH_o==9999.0].shape
print vuelos[vuelos.FRTH_d==9999.0].shape
print vuelos[vuelos.TSUN_o==9999.0].shape
print vuelos[vuelos.TSUN_d==9999.0].shape
print vuelos[vuelos.WDMV_o==9999.0].shape
print vuelos[vuelos.WDMV_d==9999.0].shape

(212698, 88)
(93590, 88)
(80587, 88)
(80129, 88)
(66248, 88)
(29922, 88)
(5314, 88)
(212687, 88)
(212691, 88)
(45673, 88)
(59212, 88)
(209910, 88)
(210240, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212687, 88)
(212691, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)
(212698, 88)


In [104]:
#todas las estaciones
stationsdf.to_csv('stationsAll.csv',sep=',', index=False)

In [141]:
stationsTAVG =  weatherdf[weatherdf["type"]=="TAVG"]
stationsTAVG = stationsTAVG.sort_values(by=['id'])
stationsTMAX =  weatherdf[weatherdf["type"]=="TMAX"]
stationsTMAX = stationsTMAX.sort_values(by=['id'])
stationsTMIN =  weatherdf[weatherdf["type"]=="TMIN"]
stationsTMIN = stationsTMIN.sort_values(by=['id'])
stationsPRCP =  weatherdf[weatherdf["type"]=="PRCP"]
stationsPRCP = stationsPRCP.sort_values(by=['id'])



In [140]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)


In [108]:
def getLatstation(row):
    row['lat'] = stationsdf[stationsdf['id']==row['id']]['lat'].values[0]
    row['lng'] = stationsdf[stationsdf['id']==row['id']]['lng'].values[0]
    return row


def obtenerEstacionBoard(row):
    lat = row['board_lat']
    lon = row['board_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lista), coor)
    EMC[0] = round(EMC[0],3)
    EMC[1] = round(EMC[1],3)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[(stationsdf.lat==EMC[0])|(stationsdf.lng==EMC[1])]
    EMC = aux['id']
    s = EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row


def aplicarEstacionesOrigen(row):
    row['board_stationid_o'] = coordenadasOrigen[(coordenadasOrigen.board_lat == row['board_lat'])&(coordenadasOrigen.board_lon == row['board_lon'])]['stationid'].values[0]
    return row

def obtenerEstacionOff(row):
    lat = row['off_lat']
    lon = row['off_lon']
    coor = [lat,lon]
    EMC= closest(np.asarray(lista), coor)
    EMC[0] = round(EMC[0],3)
    EMC[1] = round(EMC[1],3)
    ##obtenemos el id de la estacion de board
    aux = stationsdf[(stationsdf.lat==EMC[0])|(stationsdf.lng==EMC[1])]
    EMC = aux['id']
    s = EMC.to_string()
    stationid = str(s)
    row['stationid'] = stationid.split()[1]
    return row

def aplicarEstacionesDestino(row):
    row['off_stationid_d'] = coordenadasDestino[(coordenadasDestino.off_lat == row['off_lat'])&(coordenadasDestino.off_lon == row['off_lon'])]['stationid'].values[0]
    return row

def roundF(row):
    row['lat'] = round(row['lat'],3)
    
    return row

def roundFF(row):
    row['lng'] = round(row['lng'],3)
    return row



In [142]:
def obtenerTAVG_o2(row):
    board_station = row['board_stationid_o']
    fechaAux = tratarFecha(row['actual_time_of_departure'])
    x = stationsTAVG[stationsTAVG.date == fechaAux]
    if x.empty:
        pass
    else:
        x= x[x.id == board_station]
        if x.empty:
            pass
        else:
            row['TAVG_o'] = float(x.Value1.values[0])/10

        
    del x
    return row
def obtenerTAVG_d2(row):
    off_station = row['off_stationid_d']
    fechaAux = tratarFecha(row['actual_time_of_departure'])
    x = stationsTAVG[stationsTAVG.date == fechaAux]
    if x.empty:
        pass
    else:
        x= x[x.id == off_station]
        if x.empty:
            pass
        else:
            row['TAVG_d'] = float(x.Value1.values[0])/10

        
    del x
    return row
def obtenerTMIN_o2(row):
    board_station = row['board_stationid_o']
    fechaAux = tratarFecha(row['actual_time_of_departure'])
    x = stationsTMIN[stationsTMIN.date == fechaAux]
    if x.empty:
        pass
    else:
        x= x[x.id == board_station]
        if x.empty:
            pass
        else:
            row['TMIN_o'] = float(x.Value1.values[0])/10

        
    del x
    return row
def obtenerTMIN_d2(row):
    off_station = row['off_stationid_d']
    fechaAux = tratarFecha(row['actual_time_of_departure'])
    x = stationsTMIN[stationsTMIN.date == fechaAux]
    if x.empty:
        pass
    else:
        x= x[x.id == off_station]
        if x.empty:
            pass
        else:
            row['TMIN_d'] = float(x.Value1.values[0])/10

        
    del x
    return row





In [143]:
def functionGest_TAVG_d(year, month, day):
    start = time.time()
    vuelosSinTratar = vuelos[vuelos.TAVG_d==9999.0]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    x = x.apply(lambda x:obtenerTAVG_d2(x),axis = 1)
    
    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TAVG_d'] = x.set_index(['index'])['TAVG_d'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_d']).values

    vuelos['TAVG_d'] = vuelosSinTratar.set_index(['index'])['TAVG_d'].combine_first(vuelos.set_index(['index'])['TAVG_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

def functionGest_TAVG_o(year, month, day):
    start = time.time()
    vuelosSinTratar = vuelos[vuelos.TAVG_o==9999.0]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    x = x.apply(lambda x:obtenerTAVG_o2(x),axis = 1)
    
    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TAVG_o'] = x.set_index(['index'])['TAVG_o'].combine_first(vuelosSinTratar.set_index(['index'])['TAVG_o']).values
    vuelos['TAVG_o'] = vuelosSinTratar.set_index(['index'])['TAVG_o'].combine_first(vuelos.set_index(['index'])['TAVG_o']).values
    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)
def functionGest_TMIN_d(year, month, day):
    start = time.time()
    vuelosSinTratar = vuelos[vuelos.TMIN_d==9999.0]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    x = x.apply(lambda x:obtenerTAVG_d2(x),axis = 1)
    
    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_d'] = x.set_index(['index'])['TMIN_d'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_d']).values

    vuelos['TMIN_d'] = vuelosSinTratar.set_index(['index'])['TMIN_d'].combine_first(vuelos.set_index(['index'])['TMIN_d']).values

    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

def functionGest_TMIN_o(year, month, day):
    start = time.time()
    vuelosSinTratar = vuelos[vuelos.TMIN_o==9999.0]
     
    x = vuelosSinTratar[vuelosSinTratar['anyoSalida']==year]
    x = x[x['mesSalida']==month]
    x = x[x['diaSalida']==day]
    x = x.apply(lambda x:obtenerTAVG_o2(x),axis = 1)
    
    pd.options.mode.chained_assignment = None  # default='warn'
    vuelosSinTratar['TMIN_o'] = x.set_index(['index'])['TMIN_o'].combine_first(vuelosSinTratar.set_index(['index'])['TMIN_o']).values
    vuelos['TMIN_o'] = vuelosSinTratar.set_index(['index'])['TMIN_o'].combine_first(vuelos.set_index(['index'])['TMIN_o']).values
    
    #vuelos.to_csv('vuelos.csv', sep=',', index=False)
    end = time.time()
    print(end - start)

In [120]:
#TAVG----------------------FALTA AJUSTARLO PARA CADA VARIABLE
if year=='2017':
    if os.path.exists('listaTAVG2017.csv'):
        lista = pd.read_csv('listaTAVG2017.csv',sep = ',')
        del lista['id']
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2017.csv', sep=',', index=False)
        del lista['id']
if year=='2016':
    if os.path.exists('listaTAVG2016.csv'):
        lista = pd.read_csv('listaTAVG2016.csv',sep = ',')
        del lista['id']        
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2016.csv', sep=',', index=False)
        del lista['id']
if year=='2015':
    if os.path.exists('listaTAVG2015.csv'):
        lista = pd.read_csv('listaTAVG2015.csv',sep = ',')
        del lista['id']
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2015.csv', sep=',', index=False)
        del lista['id']
        
if year=='2014':
    if os.path.exists('listaTAVG2014.csv'):
        lista = pd.read_csv('listaTAVG2014.csv',sep = ',')
        del lista['id']
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2014.csv', sep=',', index=False)
        del lista['id']      
if year=='2013':
    if os.path.exists('listaTAVG2013.csv'):
        lista = pd.read_csv('listaTAVG2013.csv',sep = ',')
        del lista['id']          
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2013.csv', sep=',', index=False)
        del lista['id']      
if year=='2012':
    if os.path.exists('listaTAVG2012.csv'):
        lista = pd.read_csv('listaTAVG2012.csv',sep = ',')
        del lista['id']
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2012.csv', sep=',', index=False)
        del lista['id']        
if year=='2011':
    if os.path.exists('listaTAVG2011.csv'):
        lista = pd.read_csv('listaTAVG2011.csv',sep = ',')
        del lista['id']
    else:
        lista = stationsTAVG['id'].unique()
        lista = pd.DataFrame(lista)
        lista.columns = ['id']
        lista = lista.apply(lambda x:getLatstation(x),axis=1)
        lista.to_csv('listaTAVG2011.csv', sep=',', index=False)
        del lista['id']  

In [121]:
if 'off_stationid_d' in vuelos:
    pass
else:
    stationsdf = stationsdf.apply(lambda x: roundF(x),axis = 1)
    stationsdf = stationsdf.apply(lambda x: roundFF(x),axis = 1)

    coordenadasOrigen = vuelos[['board_lat','board_lon']]
    coordenadasDestino = vuelos[['off_lat','off_lon']]
    coordenadasOrigen = coordenadasOrigen.drop_duplicates(subset=['board_lat', 'board_lon'])
    coordenadasDestino = coordenadasDestino.drop_duplicates(subset=['off_lat', 'off_lon'])
    coordenadasOrigen = coordenadasOrigen.apply(lambda x:obtenerEstacionBoard(x),axis = 1)
    coordenadasDestino = coordenadasDestino.apply(lambda x:obtenerEstacionOff(x),axis = 1)
    vuelos = vuelos.apply(lambda x: aplicarEstacionesOrigen(x),axis = 1)
    vuelos = vuelos.apply(lambda x: aplicarEstacionesDestino(x),axis = 1)

In [122]:
year2 = int(year)
for mes in range(1,13):
    print 'empiezo el mes', mes
    mesw = str(mes)
    for dia in range(1,32):
        print 'empiezo el dia - ' ,dia
        diaw = str(dia)
        if dia < 10:
            diaw = '0'+diaw

        functionGest(year2,mes,dia)

empiezo el mes 1
empiezo el dia -  1
0.4390001297
empiezo el dia -  2
0.379999876022
empiezo el dia -  3
0.40299987793
empiezo el dia -  4
0.366000175476
empiezo el dia -  5
0.414000034332
empiezo el dia -  6
0.420000076294
empiezo el dia -  7
0.412999868393
empiezo el dia -  8
0.43700003624
empiezo el dia -  9
0.47000002861
empiezo el dia -  10
0.415999889374
empiezo el dia -  11
0.47100019455
empiezo el dia -  12
0.460999965668
empiezo el dia -  13
0.442999839783
empiezo el dia -  14
0.444000005722
empiezo el dia -  15
0.467000007629
empiezo el dia -  16
0.427999973297
empiezo el dia -  17
0.348999977112
empiezo el dia -  18
0.40499997139
empiezo el dia -  19
0.407999992371
empiezo el dia -  20
0.427999973297
empiezo el dia -  21
0.423000097275
empiezo el dia -  22
0.432999849319
empiezo el dia -  23
0.398000001907
empiezo el dia -  24
0.366999864578
empiezo el dia -  25
0.432000160217
empiezo el dia -  26
0.424000024796
empiezo el dia -  27
0.382000207901
empiezo el dia -  28
0.4229

0.411999940872
empiezo el dia -  12
0.445999860764
empiezo el dia -  13
0.430999994278
empiezo el dia -  14
0.425000190735
empiezo el dia -  15
0.376999855042
empiezo el dia -  16
0.461999893188
empiezo el dia -  17
0.443000078201
empiezo el dia -  18
0.403000116348
empiezo el dia -  19
0.433000087738
empiezo el dia -  20
0.46599984169
empiezo el dia -  21
0.43799996376
empiezo el dia -  22
0.384999990463
empiezo el dia -  23
0.463000059128
empiezo el dia -  24
0.453000068665
empiezo el dia -  25
0.418999910355
empiezo el dia -  26
0.427000045776
empiezo el dia -  27
0.458999872208
empiezo el dia -  28
0.398000001907
empiezo el dia -  29
0.381000041962
empiezo el dia -  30
0.459999799728
empiezo el dia -  31
0.444999933243
empiezo el mes 9
empiezo el dia -  1
0.432999849319
empiezo el dia -  2
0.430000066757
empiezo el dia -  3
0.438999891281
empiezo el dia -  4
0.43499994278
empiezo el dia -  5
0.393000125885
empiezo el dia -  6
0.423999786377
empiezo el dia -  7
0.429000139236
empiez

In [138]:
vuelos.to_csv('vuelos.csv', sep=',', index=False)
vuelosSinTAVG_d = vuelos[vuelos.TAVG_d==9999.0]
vuelosSinTAVG_d.shape

(29922, 88)